In [ ]:
# google drive connect
Copied_path = '/content/drive/MyDrive/Colab Notebooks/MyPaper/GlocalSTN' # Paste target directory here
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir(Copied_path)

Mounted at /content/drive


In [ ]:
# m0_Ent

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m0_Ent.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
'''
# DST Network/ilayer.py 이건 ST resnet 쪽에 있던 ilayer,똑같은 거 같음


from keras import backend as K
from keras.engine.topology import Layer
# from keras.layers import Dense
import numpy as np


class iLayer(Layer):
    def __init__(self, **kwargs):
        # self.output_dim = output_dim
        super(iLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        initial_weight_value = np.random.random(input_shape[1:])
        self.W = K.variable(initial_weight_value)
        self.trainable_weights = [self.W]

    def call(self, x, mask=None):
        return x * self.W

    def get_output_shape_for(self, input_shape):
        return input_shape
'''


'\n# DST Network/ilayer.py 이건 ST resnet 쪽에 있던 ilayer,똑같은 거 같음\n\n\nfrom keras import backend as K\nfrom keras.engine.topology import Layer\n# from keras.layers import Dense\nimport numpy as np\n\n\nclass iLayer(Layer):\n    def __init__(self, **kwargs):\n        # self.output_dim = output_dim\n        super(iLayer, self).__init__(**kwargs)\n\n    def build(self, input_shape):\n        initial_weight_value = np.random.random(input_shape[1:])\n        self.W = K.variable(initial_weight_value)\n        self.trainable_weights = [self.W]\n\n    def call(self, x, mask=None):\n        return x * self.W\n\n    def get_output_shape_for(self, input_shape):\n        return input_shape\n'

In [ ]:
# DST Network/ilayer.py

from keras import backend as K
#from keras.engine.topology import Layer
#from tensorflow.keras.layers import Layer
from keras.layers import Layer
# from keras.layers import Dense
import numpy as np


class iLayer(Layer):
    def __init__(self, **kwargs):
        # self.output_dim = output_dim
        super(iLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        initial_weight_value = np.random.random(input_shape[1:])
        self.W = K.variable(initial_weight_value)
        #self.trainable_weights = [self.W]
        self.trainable_weight = [self.W]

    def call(self, x, mask=None):
        return x * self.W

    def get_output_shape_for(self, input_shape):
        return input_shape

In [ ]:
# "metrics.py"

# import numpy as np
from keras import backend as K

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def root_mean_square_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5


# aliases
mse = MSE = mean_squared_error
# rmse = RMSE = root_mean_square_error


def masked_mean_squared_error(y_true, y_pred):
    idx = (y_true > 1e-6).nonzero()
    return K.mean(K.square(y_pred[idx] - y_true[idx]))

def masked_rmse(y_true, y_pred):
    return masked_mean_squared_error(y_true, y_pred) ** 0.5


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))

def mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


threshold=0.05

def mean_absolute_percentage_error(y_true, y_pred):
    return K.mean( K.abs(y_pred-y_true) / K.maximum(K.cast(threshold,'float32'),y_true+1.0) )

def mape(y_true, y_pred):
    return K.mean( K.abs(y_pred-y_true) / K.maximum(K.cast(threshold,'float32'),y_true+1.0) )



In [ ]:
'''
    ST-ResNet: Deep Spatio-temporal Residual Networks
'''
#!pip install keras-layer-normalization
from keras.layers.convolutional import Conv2D
from __future__ import print_function
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Reshape
)
from keras.layers.convolutional import Convolution2D
#from keras.layers.normalization import BatchNormalization
from keras.layers import Input,Activation,Dropout,BatchNormalization,AveragePooling2D,ZeroPadding2D,Multiply, Add
#from keras.layers import concatenate
#from tensorflow.keras.layers.normalization import BatchNormalization
from keras.models import Model
#from keras.utils.visualize_util import plot


def _shortcut(input, residual):
    #return merge([input, residual], mode='sum')
    return Add()([input,residual])



def _bn_relu_conv(nb_filter, nb_row, nb_col, subsample=(1, 1), bn=False):
    def f(input):
        if bn:
            input = BatchNormalization(mode=0, axis=1)(input)
        activation = Activation('relu')(input)
        #return Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample, border_mode="same")(activation)
        return Conv2D(filters=nb_filter, kernel_size=(nb_row, nb_col), strides=(1,1), padding="same")(activation)

    return f


def _residual_unit(nb_filter, init_subsample=(1, 1)):
    def f(input):
        residual = _bn_relu_conv(nb_filter, 3, 3)(input)
        residual = _bn_relu_conv(nb_filter, 3, 3)(residual)
        return _shortcut(input, residual)
    return f


def ResUnits(residual_unit, nb_filter, repetations=1):
    def f(input):
        for i in range(repetations):
            init_subsample = (1, 1)
            input = residual_unit(nb_filter=nb_filter,init_subsample=init_subsample)(input)
        return input
    return f


def stresnet(c_conf=(3, 2, 32, 32), p_conf=(3, 2, 32, 32), t_conf=(3, 2, 32, 32), external_dim=8, nb_residual_unit=3, CF=64):
    '''
    C - Temporal Closeness
    P - Period
    T - Trend
    conf = (len_seq, nb_flow, map_height, map_width)
    external_dim
    '''

    # main input
    main_inputs = []
    outputs = []
    for conf in [c_conf, p_conf, t_conf]:
        if conf is not None:
            len_seq, nb_flow, map_height, map_width = conf
            input = Input(shape=(nb_flow * len_seq, map_height, map_width))
            main_inputs.append(input)
            # Conv1
            #conv1 = Convolution2D(nb_filter=CF, nb_row=3, nb_col=3, border_mode="same")(input)
            conv1 = Conv2D(filters=CF, kernel_size=(3,3),padding="same")(input)
            # [nb_residual_unit] Residual Units
            residual_output = ResUnits(_residual_unit, nb_filter=CF,
                              repetations=nb_residual_unit)(conv1)
            # Conv2
            activation = Activation('relu')(residual_output)
            #conv2 = Convolution2D(nb_filter=nb_flow, nb_row=3, nb_col=3, border_mode="same")(activation)
            conv2 = Conv2D(filters=nb_flow, kernel_size=(3,3),padding="same")(activation)
            outputs.append(conv2)

    # parameter-matrix-based fusion
    if len(outputs) == 1:
        main_output = outputs[0]
    else:
        #from DST_network.ilayer import iLayer
        new_outputs = []
        for output in outputs:
            new_outputs.append(iLayer()(output))
        #main_output = merge(new_outputs, mode='sum')
        main_output = Add()(new_outputs)

    # fusing with external component
    if external_dim != None and external_dim > 0:
        # external input
        external_input = Input(shape=(external_dim,))
        main_inputs.append(external_input)
        embedding = Dense(output_dim=10)(external_input)
        embedding = Activation('relu')(embedding)
        h1 = Dense(output_dim=nb_flow * map_height * map_width)(embedding)
        activation = Activation('relu')(h1)
        external_output = Reshape((nb_flow, map_height, map_width))(activation)
        main_output = merge([main_output, external_output], mode='sum')
    else:
        print('external_dim:', external_dim)

    main_output = Activation('tanh')(main_output)
    model = Model(inputs=main_inputs, outputs=main_output)

    return model
'''
if __name__ == '__main__':
    model = stresnet(external_dim=28, nb_residual_unit=12)
    #plot(model, to_file='ST-ResNet.png', show_shapes=True)
    model.summary()
'''

"\nif __name__ == '__main__':\n    model = stresnet(external_dim=28, nb_residual_unit=12)\n    #plot(model, to_file='ST-ResNet.png', show_shapes=True)\n    model.summary()\n"

In [ ]:

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K


#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 4  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=10


In [ ]:

print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))



loading data...
all_data shape:  (797, 1, 14, 12)
max= 33.0  min= 0.0
mean= -0.9878032883051704  variance= 0.06483403736343217
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.002324 0.048211 0.010578]
RMSE  MAE
[[0.0482 0.0106]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0048 0.0011]
totally cost 35.27445697784424
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.002225 0.047165 0.012295]
RMSE  MAE
[[0.0482 0.0106]
 [0.0472 0.0123]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0095 0.0023]
totally cost 19.441701412200928
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.002239 0.047322 0.011563]
RMSE  MAE
[[0.0482 0.0106]
 [0.0472 0.0123]
 [0.0473 0.0116]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.   

In [ ]:
# m1_Col

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m1_Col.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 13.0  min= 0.0
mean= -0.9942343311226624  variance= 0.04226600741012836
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.000836 0.02892  0.004363]
RMSE  MAE
[[0.0289 0.0044]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0029 0.0004]
totally cost 19.63086175918579
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.000834 0.028876 0.005142]
RMSE  MAE
[[0.0289 0.0044]
 [0.0289 0.0051]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0058 0.001 ]
totally cost 19.730453491210938
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.000832 0.028847 0.005577]
RMSE  MAE
[[0.0289 0.0044]
 [0.0289 0.0051]
 [0.0288 0.0056]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.   

In [ ]:
# m2_Ev

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m2_Ev.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 1.0  min= 0.0
mean= -0.9998356933739618  variance= 0.018126948320356582
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.000213 0.014581 0.000138]
RMSE  MAE
[[0.0146 0.0001]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0015 0.    ]
totally cost 19.482852935791016
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.000213 0.014582 0.000182]
RMSE  MAE
[[0.0146 0.0001]
 [0.0146 0.0002]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0029 0.    ]
totally cost 19.925350189208984
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.000213 0.014581 0.000142]
RMSE  MAE
[[0.0146 0.0001]
 [0.0146 0.0002]
 [0.0146 0.0001]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.  

In [ ]:
# m3_Food

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m3_Food.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 63.0  min= 0.0
mean= -0.9811303442330509  variance= 0.07453364279577186
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.003332 0.057726 0.017926]
RMSE  MAE
[[0.0577 0.0179]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0058 0.0018]
totally cost 20.01195192337036
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.003646 0.060383 0.014674]
RMSE  MAE
[[0.0577 0.0179]
 [0.0604 0.0147]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0118 0.0033]
totally cost 19.73185634613037
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.003369 0.058041 0.018474]
RMSE  MAE
[[0.0577 0.0179]
 [0.0604 0.0147]
 [0.058  0.0185]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.    

In [ ]:
# m4_Night

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m4_Night.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 3.0  min= 0.0
mean= -0.9968283842185973  variance= 0.048270752295892394
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.001673 0.040898 0.003048]
RMSE  MAE
[[0.0409 0.003 ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0041 0.0003]
totally cost 20.73455500602722
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001677 0.040954 0.002414]
RMSE  MAE
[[0.0409 0.003 ]
 [0.041  0.0024]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0082 0.0005]
totally cost 19.617950439453125
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001677 0.040947 0.00281 ]
RMSE  MAE
[[0.0409 0.003 ]
 [0.041  0.0024]
 [0.0409 0.0028]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.   

In [ ]:
# m5_Outdoor

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m5_Outdoor.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 130.0  min= 0.0
mean= -0.9962962069298331  variance= 0.0168414251671113
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.000116 0.010757 0.00276 ]
RMSE  MAE
[[0.0108 0.0028]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0011 0.0003]
totally cost 19.54152512550354
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.000115 0.010744 0.002763]
RMSE  MAE
[[0.0108 0.0028]
 [0.0107 0.0028]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0022 0.0006]
totally cost 19.739707231521606
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.000116 0.010772 0.00277 ]
RMSE  MAE
[[0.0108 0.0028]
 [0.0107 0.0028]
 [0.0108 0.0028]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.   

In [ ]:
# m6_Pro

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m6_Pro.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 35.0  min= 0.0
mean= -0.985686545634565  variance= 0.052230389053797795
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.001645 0.040562 0.01551 ]
RMSE  MAE
[[0.0406 0.0155]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0041 0.0016]
totally cost 19.594473600387573
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001698 0.041201 0.013393]
RMSE  MAE
[[0.0406 0.0155]
 [0.0412 0.0134]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0082 0.0029]
totally cost 20.32574486732483
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.00168  0.040982 0.016089]
RMSE  MAE
[[0.0406 0.0155]
 [0.0412 0.0134]
 [0.041  0.0161]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.   

In [ ]:
# m7_Res

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m7_Res.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 4.0  min= 0.0
mean= -0.9962097448766206  variance= 0.04684482697013769
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.001764 0.041994 0.003857]
RMSE  MAE
[[0.042  0.0039]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0042 0.0004]
totally cost 19.745365142822266
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001764 0.042004 0.003344]
RMSE  MAE
[[0.042  0.0039]
 [0.042  0.0033]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0084 0.0007]
totally cost 19.214434385299683
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001764 0.042006 0.003614]
RMSE  MAE
[[0.042  0.0039]
 [0.042  0.0033]
 [0.042  0.0036]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.  

In [ ]:
# m8_Shop

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m8_Shop.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 146.0  min= 0.0
mean= -0.9895630507750447  variance= 0.04576444793426465
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.0013   0.036053 0.008593]
RMSE  MAE
[[0.0361 0.0086]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0036 0.0009]
totally cost 19.90634822845459
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001281 0.035795 0.008837]
RMSE  MAE
[[0.0361 0.0086]
 [0.0358 0.0088]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0072 0.0017]
totally cost 19.871694803237915
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.001225 0.035003 0.01161 ]
RMSE  MAE
[[0.0361 0.0086]
 [0.0358 0.0088]
 [0.035  0.0116]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.   

In [ ]:
# m9_Tra

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data2/m9_Tra.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)



print("loading data...")
#X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)
X_train,T_train,Y_train,X_test,T_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

R_N = 4   # number of residual units

#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
#from DST_network.STResNet import stresnet
#import DST_network.metrics as metrics
#import PPT3_network.metrics as metrics

def build_model(external_dim,CFN):
  c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
  p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
  t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

  model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

  adam = Adam(lr=lr)
  model.compile(loss='mse', optimizer=adam, metrics=[rmse,mae])
  #model.summary()
  #from keras.utils.visualize_util import plot
  #plot(model, to_file='model.png', show_shapes=True)
  return model


CF=64

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])
count_sum=iterate_num

import time

count=0


for iterate_index in range(iterate_num):
  count=count+1
  iterate=iterate_loop[iterate_index]

  time_start=time.time()

  #F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

  model = build_model(external_dim=False,CFN=CF)
  '''
  model_checkpoint=ModelCheckpoint(
      filepath=F,
      monitor='val_rmse',
      verbose=1,
      save_best_only=True,
      save_weights_only=False,
      mode='min',
      period=1)
  '''
  print('=' * 10)
  print("training model...")
  history = model.fit(X_train, Y_train,
                      epochs=epoch,
                      batch_size=batch_size,
                      validation_split=0.1,
                      #callbacks=[model_checkpoint],
                      verbose=0)

  #print('=' * 10)
  #print('evaluating using the model that has the best loss on the valid set')
  #model.load_weights(F)

  score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
  print('              mse     rmse    mae')
  print('Train score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
  print('Test  score:',end=' ')
  np.set_printoptions(precision=6, suppress=True)
  print(np.array(score))

  RMSE[iterate_index,0]=score[1]
  MAE [iterate_index,0]=score[2]

  for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

  np.set_printoptions(precision=4, suppress=True)
  print('RMSE  MAE')
  print(for_show)

  for_show=np.mean(for_show,axis=0)
  print('RMSE  MAE')
  print(for_show)

  #np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

  time_end=time.time()

  print('totally cost',time_end-time_start)
  print(str(count)+'/'+str(count_sum))




loading data...
all_data shape:  (797, 1, 14, 12)
max= 132.0  min= 0.0
mean= -0.9752139154585828  variance= 0.08477867360777425
number_of_skip_hours: 224
len_train=461
len_test =112
external_dim: False
training model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


              mse     rmse    mae
Train score: Test  score: [0.004699 0.068548 0.02173 ]
RMSE  MAE
[[0.0685 0.0217]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0069 0.0022]
totally cost 20.038719654083252
1/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.004727 0.068751 0.021249]
RMSE  MAE
[[0.0685 0.0217]
 [0.0688 0.0212]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0137 0.0043]
totally cost 20.320078134536743
2/10
external_dim: False
training model...
              mse     rmse    mae
Train score: Test  score: [0.004341 0.065883 0.022062]
RMSE  MAE
[[0.0685 0.0217]
 [0.0688 0.0212]
 [0.0659 0.0221]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.     0.    ]
 [0.  